# Анализ потребительской лояльности клиентов телекоммуникационной компании

Заказчик исследования — крупная телекоммуникационная компания, оказывающая услуги на территории всего СНГ.

**Цель проекта:** определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России.

**Содержание**
1. Предобработка данных
2. Ссылки на дашборд и презентацию

## Предобработка данных

Импортируем необходимые библиотеки.

In [1]:
import pandas as pd
from sqlalchemy import create_engine

Подключимся нужным таблицам из базы данных.

In [2]:
path_to_db = 'C:/Documents/Yandex Practicum/Projects/3. Анализ потребительской лояльности клиентов телекоммуникационной компании/telecomm_csi.db'
engine = create_engine (f'sqlite:///{path_to_db}', echo = False)
query = """SELECT user.user_id,
                  user.lt_day,
                  CASE
                      WHEN user.lt_day <= 365 THEN 'новый'
                      WHEN user.lt_day > 365 THEN 'старый'
                  END AS is_new,
                  user.age,
                  CASE 
                      WHEN user.gender_segment = 1 THEN 'женщина'
                      WHEN user.gender_segment = 0 THEN 'мужчина'
                  END AS gender_segment,
                  user.os_name,
                  user.cpe_type_name,
                  loc.country,
                  loc.city,
                  age.title AS age_segment,
                  traffic.title AS traffic_segment,
                  life.title AS lifetime_segment,
                  user.nps_score,
                  CASE 
                      WHEN user.nps_score < 7 THEN 'критик'
                      WHEN user.nps_score > 6 AND user.nps_score < 9 THEN 'нейтрал'
                      WHEN user.nps_score > 8 THEN 'сторонник'
                  END AS nps_group
           FROM user
           JOIN location AS loc ON user.location_id = loc.location_id
           JOIN age_segment AS age ON user.age_gr_id = age.age_gr_id
           JOIN traffic_segment AS traffic ON user.tr_gr_id = traffic.tr_gr_id
           JOIN lifetime_segment AS life ON user.lt_gr_id = life.lt_gr_id;"""
data = pd.read_sql (query, engine)
data.sample (5)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           502493 non-null  object 
 1   lt_day            502493 non-null  int64  
 2   is_new            502493 non-null  object 
 3   age               501939 non-null  float64
 4   gender_segment    501192 non-null  object 
 5   os_name           502493 non-null  object 
 6   cpe_type_name     502493 non-null  object 
 7   country           502493 non-null  object 
 8   city              502493 non-null  object 
 9   age_segment       502493 non-null  object 
 10  traffic_segment   502493 non-null  object 
 11  lifetime_segment  502493 non-null  object 
 12  nps_score         502493 non-null  int64  
 13  nps_group         502493 non-null  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 53.7+ MB


Проверим пропуски в данных.

In [3]:
data.isna().sum()

user_id                0
lt_day                 0
is_new                 0
age                  554
gender_segment      1301
os_name                0
cpe_type_name          0
country                0
city                   0
age_segment            0
traffic_segment        0
lifetime_segment       0
nps_score              0
nps_group              0
dtype: int64

Имеются пропуски в колонках `age`, `gender_segment`. Их объём не превышает 1% от всего объёма данных - от них можно избавиться. Проверим также наличие дубликатов в датасете.

In [4]:
data = data.dropna()
data.duplicated().sum()

0

Также стоит проверить дубликаты без колонки `user_id`, т.к. значения в ней уникальны.

In [5]:
data.loc [:, data.columns != 'user_id'].duplicated().sum()

577

Объём дубликатов также очень мал, очистим датасет.

In [6]:
data = data.drop_duplicates().reset_index (drop = True)

Сохраним получившийся датасет в csv-файл и приступим к созданию дашборда.

In [7]:
data.to_csv ('C:/Documents/Yandex Practicum/Projects/3. Анализ потребительской лояльности клиентов телекоммуникационной компании/data.csv', sep = ',', index = False)

## Ссылки на дашборд и презентацию

**Ссылка на дашборд:** https://public.tableau.com/app/profile/artyom1313/viz/Analysisofcustomerloyalty/Dashboard
**Ссылка на презентацию:** https://disk.yandex.ru/i/lapagiYyQ6LUxA